# Домашнее задание к лекции "Рекуррентные сети" 

Задание 1.
Обучите нейронную сеть решать шифр Цезаря.

Что необходимо сделать:

    Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
    Сделать нейронную сеть
    Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
    Проверить качество


In [50]:
#Библиотеки
import pandas as pd
import time
import torch

In [51]:
word = 'лопата'

In [52]:
# Функция кодирующее слово
def encod_cesar(word, move=2):
    alphabet = list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
    word_list = list(word.lower())
    new_word_list = []
    
    for char in word_list:
        ind = alphabet.index(char) + move

        if ind >= len(alphabet):
            new_char = alphabet[ind - len(alphabet)]
        
        else:
            new_char = alphabet[ind]
        
        new_word_list.append(new_char)
    
    return ''.join(new_word_list)

    

In [53]:
# Функция декодирующее слово
def decod_cesar(word, move=2):
    alphabet = list('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
    word_list = list(word.lower())
    new_word_list = []
    
    for char in word_list:
        ind = alphabet.index(char) - move

        if ind < 0:
            new_char = alphabet[ind + len(alphabet)]
        
        else:
            new_char = alphabet[ind]
        
        new_word_list.append(new_char)
    
    return ''.join(new_word_list)


In [54]:
new_word = encod_cesar(word)

In [55]:
print(decod_cesar(new_word))

лопата


In [56]:
# делаем массивы с данными

CHARS = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [67]:
MAX_LEN = 50
X = torch.zeros((len(CHARS), 15), dtype=int)
for i in range(len(word)):
    for j, w in enumerate(word[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [68]:
# Модель
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(len(CHAR_TO_INDEX), 28)
        self.rnn = torch.nn.RNN(28, 128, batch_first=True)
        self.linear= torch.nn.Linear(128, len(INDEX_TO_CHAR))
        
    def forward(self, word, state=None):
        embed = self.embed(word)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out
        

In [69]:
model = Network()

In [70]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)


In [72]:
import pdb

# Обучение
for ep in range(10):
#     pdb.set_trace()
    start = time.time()
    train_loss = 0.
    train_passed = 0
    print(int(len(X) / 100))

    for i in range(int(len(X) / 100)):
        print('test')
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1
    
    print(train_passed)

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))

# Обучение не пошло... Похоже не до конца понял как заводить рекуррентные нейроные сети. Но в чем ошибка не пойму


0
0


ZeroDivisionError: float division by zero

Задание 2.
Выполнить практическую работу из лекционного ноутбука.

    Построить RNN-ячейку на основе полносвязных слоев
    Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”


In [13]:
# данные

df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count
0,0,10368,35,29,"Lisa Simpson: Maggie, look. What's that?",235000,True,9,5.0,Lisa Simpson,Simpson Home,"Maggie, look. What's that?",maggie look whats that,4.0
1,1,10369,35,30,Lisa Simpson: Lee-mur. Lee-mur.,237000,True,9,5.0,Lisa Simpson,Simpson Home,Lee-mur. Lee-mur.,lee-mur lee-mur,2.0
2,2,10370,35,31,Lisa Simpson: Zee-boo. Zee-boo.,239000,True,9,5.0,Lisa Simpson,Simpson Home,Zee-boo. Zee-boo.,zee-boo zee-boo,2.0
3,3,10372,35,33,Lisa Simpson: I'm trying to teach Maggie that ...,245000,True,9,5.0,Lisa Simpson,Simpson Home,I'm trying to teach Maggie that nature doesn't...,im trying to teach maggie that nature doesnt e...,24.0
4,4,10374,35,35,"Lisa Simpson: It's like an ox, only it has a h...",254000,True,9,5.0,Lisa Simpson,Simpson Home,"It's like an ox, only it has a hump and a dewl...",its like an ox only it has a hump and a dewlap...,18.0


In [14]:
phrases = df['normalized_text'].tolist()
phrases[:10]

['maggie look whats that',
 'lee-mur lee-mur',
 'zee-boo zee-boo',
 'im trying to teach maggie that nature doesnt end with the barnyard i want her to have all the advantages that i didnt have',
 'its like an ox only it has a hump and a dewlap hump and dew-lap hump and dew-lap',
 'you know his blood type how romantic',
 'oh yeah whats my shoe size',
 'ring',
 'yes dad',
 'ooh look maggie what is that do-dec-ah-edron dodecahedron']

In [15]:
text = [[c for c in ph] for ph in phrases if type(ph) is str]

In [16]:
CHARS = set('abcdefghijklmnopqrstuvwxyz ')
INDEX_TO_CHAR = ['none'] + [w for w in CHARS]
CHAR_TO_INDEX = {w: i for i, w in enumerate(INDEX_TO_CHAR)}

In [17]:
MAX_LEN = 50
X = torch.zeros((len(text), MAX_LEN), dtype=int)
for i in range(len(text)):
    for j, w in enumerate(text[i]):
        if j >= MAX_LEN:
            break
        X[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

In [45]:
# Сеть

class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(len(CHAR_TO_INDEX), 28)
        self.rnn = torch.nn.RNN(28, 128, batch_first=True)
        self.linear= torch.nn.Linear(128, len(INDEX_TO_CHAR))
        
    def forward(self, word, state=None):
        embed = self.embed(word)
        o, s = self.rnn(embed)
        out = self.linear(o)

        return out
        

In [46]:
model = Network()

In [47]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)


In [48]:
# Функция генерации последовательности

def generate_sentence():
    sentence = ['h', 'e', 'l', 'l', 'o']
    
    x = torch.zeros((1, len(sentence)), dtype=int)
    
    for j, w in enumerate(sentence):
        if j >= MAX_LEN:
            break
        
        else:
            x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])
#     s = None
    
    for i in range(MAX_LEN):
        o = model(x)
        w = torch.argmax(o[-1, -1, :], keepdim=True)
        x = torch.cat([x, w.unsqueeze(0)], axis=1)
        ww = INDEX_TO_CHAR[w]
        
        if ww == 'none':
            break
            
        sentence.append(ww)
    
    print(' '.join(sentence))
    
    
    
    

In [49]:
# Обучение и генерация фраз

for ep in range(10):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1]
        Y_batch = batch[:, 1:].flatten()

        optimizer.zero_grad()
        answers = model.forward(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    generate_sentence()

Epoch 0. Time: 6.714, Train loss: 2.095
h e l l o   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t   t
Epoch 1. Time: 6.384, Train loss: 1.814
h e l l o   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t
Epoch 2. Time: 6.527, Train loss: 1.727
h e l l o   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t
Epoch 3. Time: 6.246, Train loss: 1.677
h e l l o   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t
Epoch 4. Time: 8.806, Train loss: 1.642
h e l l o   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t
Epoch 5. Time: 8.029, Train loss: 1.617
h e l l o   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t h e   t
Epoch 6. Time: 8.762, Train loss: 1.596
h e l l o   t h e   t h e   t h e   t h e   t h e   t h e   

Но опять же почему-то не завелась...